In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import pickle
import os
from nlp_surveillance.utils.my_utils import remove_nans, check_url_validity, remove_guillemets, split_and_flatten_list, flatten_list
from nlp_surveillance.utils.text_from_url import extract_cleaned_text_from_url
from nlp_surveillance.annotator import *
from nlp_surveillance.edb_clean import get_cleaned_edb, read_minimal_cleaned_edb
from nlp_surveillance.wiki_country_parser import get_wiki_countries_df
from nlp_surveillance.who_scraper import get_annotated_2018_whos

# Testing

# WHO DF

In [ ]:
parsed_whos_df = get_annotated_2018_whos()

### EDB 

In [ ]:
edb = get_cleaned_edb()
pickle.dump(edb, open('edb.p', 'wb'))

In [ ]:
edb = pickle.load(open('edb.p', 'rb'))

## Optimize date

In [ ]:
sentences = pd.Series(np.zeros(len(date_optimization_edb)))
annos = pd.Series(np.zeros(len(date_optimization_edb)))
for i, text in enumerate(tqdm(date_optimization_edb['text'])):
    try:
        annotated = annotate(text, tiers='DateAnnotator()')
        date_spans = annotated.tiers['dates'].spans
        date_sentences = list(map(lambda x: get_sentence_from_annotated_span(x, text), date_spans))
    except Exception as e:
        print(i, e)
        date_sentences = np.nan
    sentences.iloc[i] = date_sentences
    annos.iloc[i] = annotated

In [ ]:
pickle.dump(sentences, open('sentences.p','wb'))

In [ ]:
sentences = pickle.load(open('sentences.p', 'rb'))

In [ ]:
date_optimization_edb['sentences'] = sentences
#date_optimization_edb['annotatations'] = list(map(lambda x: dates(x, raw=True), annos))

#date_optimization_edb = date_optimization_edb[date_optimization_edb['annotatations'] != Entity('dates', [])]

#date_optimization_edb['annotatations'] = date_optimization_edb['annotatations'].apply(lambda x: x[1])

edb_dates = date_optimization_edb['Datenstand für Fallzahlen gesamt*'].apply(lambda x: str(x))
date_optimization_edb['Datenstand für Fallzahlen gesamt*'] = edb_dates